In [1]:
import numpy as np
import bm3d
from pathlib import Path
import pickle
from skimage import io, img_as_float
import nibabel as nib
import os
import glob
from collections import defaultdict

File Paths

In [2]:
icip_data = "D:/VIP Cup/Dataset/ICIP_test_data_final/"
final_denoised_dataset = "D:/VIP Cup/Output dataset/test_dataset_denoised_upsampled"



Constants

In [3]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.085
oct_height = 300
oct_width = 300

Functions to check whether a tiff file/directory or not

In [4]:
def is_tiff_dir(curr_path):
    '''function to check if the current directory having any tiff files'''
    num_tiff = len(glob.glob(os.path.join(curr_path, '*.tif')))
    return num_tiff

def is_tiff(curr_path):
    '''function to check if the current file is a tiff file or not'''
    if os.path.isfile(curr_path) and (".tif" in curr_path):
        return True
    else:
        return False

Searching tiff directories

In [5]:
tiff_directories = defaultdict(int)
denoised_tiff_count = 0
original_tiff_count = 0
SIGMA = 0.085

dataset = glob.iglob(os.path.join(icip_data, '**'), recursive=True)

for i,file in enumerate(dataset):
    if is_tiff_dir(file):

        current_tiff_count = is_tiff_dir(file)
        tiff_directories[file] = current_tiff_count
        print(f"{len(tiff_directories)}th case {file} ......")
        original_tiff_count += current_tiff_count



print(f"Denoised cases: {len(tiff_directories)}")
#print(f"Denoised tiff images: {denoised_tiff_count}")
print(f"Original tiff images: {original_tiff_count}")


1th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (1) ......
2th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (10) ......
3th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (11) ......
4th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (12) ......
5th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (13) ......
6th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (14) ......
7th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (15) ......
8th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (16) ......
9th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (17) ......
10th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (18) ......
11th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (2) ......
12th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (3) ......
13th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (4) ......
14th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (5) ......
15th c

Denoising Function

In [6]:
import numpy as np
from numpy.linalg import svd
from PIL import Image
import math

def get_svd_image_from_array(A):
    # Convert the image to a NumPy array

    # Remember image size
    n = len(A)

    # Take SVD
    U, S, V = svd(A)

    threshold = 4/np.sqrt(3)
    filter = (S > threshold)*1 
    filtered_S = np.multiply(S, filter)

    # Initialize blank image
    final_img = np.zeros((n, n))
    
    for i in range(n):
        matrix = S[i]*np.outer(U[:,i],V[i])
        final_img += matrix
    
    return final_img

In [7]:
def open_noisy_image_as_array(image_path):
    image_array = img_as_float(io.imread(image_path, as_gray=True))
    return np.array(image_array)

In [8]:
def bm3d_denoise(noisy_image, npsd_sigma = SIGMA):

    noisy_oct = img_as_float(io.imread(noisy_image, as_gray=True))
    sigma = npsd_sigma
    bm3d_denoised = bm3d.bm3d(noisy_oct, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    return bm3d_denoised

In [9]:
def bm3d_denoise_from_image_array(noisy_image_array, npsd_sigma = SIGMA):

    sigma = npsd_sigma
    bm3d_denoised = bm3d.bm3d(noisy_image_array, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    return bm3d_denoised

Super Resolution functions

In [10]:
import cv2
import numpy as np

def get_bicubic_upsampled_array(image: np.array, width: int = 300, height: int = 300) -> np.array:
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_CUBIC)

def get_bilinear_upsampled_array(image: np.array, width: int = 300, height: int = 300) -> np.array:
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_LINEAR)

def get_nearest_upsampled_array(image: np.array, width: int = 300, height: int = 300) -> np.array:
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_NEAREST)

def get_lanczos_upsampled_array(image: np.array, width: int = 300, height: int = 300) -> np.array:
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_LANCZOS4)


Saving Denoised Image as a .tiff file

In [17]:
from matplotlib import pyplot as plt

def save_denoised_image_as_tiff(denoised_array, output_path):
    
    plt.imsave(output_path, denoised_array, cmap='gray')

Output path generating function

In [18]:
def generate_out_path(file, output_dir, sigma):
    '''generate output path'''
    file_name = os.path.basename(file).split(".")[0]
    case_name = os.path.basename(os.path.dirname(file))
    class_name = os.path.basename(os.path.dirname(os.path.dirname(file)))

    case_dir = os.path.join(output_dir,class_name,case_name)
    os.makedirs(case_dir, exist_ok=True)

    out_path = os.path.join(case_dir, file_name + ".tiff")
    return out_path

Denoising and super resolving Function

In [19]:
#denoising function
def denoise_and_upsample(image_path, output_path, sigma=SIGMA):
    '''function to denoise the image'''
    bm3d_denoised = bm3d_denoise(image_path, sigma)
    upsampled_img = get_bicubic_upsampled_array(bm3d_denoised, oct_width, oct_height)
    save_denoised_image_as_tiff(upsampled_img, output_path)

In [20]:
tiff_directories

defaultdict(int,
            {'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (1)': 150,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (10)': 262,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (11)': 264,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (12)': 237,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (13)': 262,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (14)': 232,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (15)': 220,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (16)': 81,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (17)': 138,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (18)': 226,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (2)': 263,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQA (3)': 293,
             'D:/VIP Cup/Dataset/ICIP_test_data_final\\RawDataQ

Denoising Pipeline

In [21]:
# denoising pipeline for denoise the tiff images in each and every tiff directories
def denoising_pipeline(tiff_directories, final_denoised_dataset, SIGMA):
    '''denoising pipeline for denoise the tiff images in each and every tiff directories'''
    count = 0

    for i, tiff_dir in enumerate(tiff_directories):
        print(f"{i}th case {tiff_dir} ......")

        tiff_files = os.listdir(tiff_dir)

        if len(tiff_files) == tiff_directories[tiff_dir]:
            for tiff_file in tiff_files:
                tiff_file_path = os.path.join(tiff_dir, tiff_file)
                output_path = generate_out_path(tiff_file_path, final_denoised_dataset, SIGMA)
                denoise_and_upsample(tiff_file_path, output_path, SIGMA)
        else:
            raise Exception(f"Number of tiff files in the directory is not equal to the number of tiff files in the dictionary in {tiff_dir}")

        print(f"{i}th case {tiff_dir} denoised ......")
        count += 1

    return count
count = 0

In [22]:
# # denoising pipeline for denoise the tiff images in each and every tiff directories
# denoised_tiff_count = denoising_pipeline(tiff_directories, final_denoised_dataset, SIGMA)
# print(f"Denoised cases: {denoised_tiff_count}")

0th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (1) ......
0th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (1) denoised ......
1th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (10) ......
1th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (10) denoised ......
2th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (11) ......
2th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (11) denoised ......
3th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (12) ......
3th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (12) denoised ......
4th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (13) ......
4th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (13) denoised ......
5th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (14) ......
5th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (14) denoised ......
6th case D:/VIP Cup/Dataset/ICIP_test_data_final\RawDataQA (15) ......
6th case D:/VIP Cup/Datas